In [1]:
import os
import yaml
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import numpy as np

In [2]:
# Change the current working directory to the project root folder
# This ensures the code works with relative paths and avoids FileNotFoundError
os.chdir("../")
%pwd

'd:\\Projects\\Apple-Disease-Classification-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    built_model_path: Path
    trained_model_path: Path    
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    params_patience: int
    params_factor: float
    params_min_lr: float
    params_verbose: int

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.logging import logger

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        model_preparation = self.config.model_preparation
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "dataset")
        create_directories([ Path(training.root_dir) ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            built_model_path=Path(model_preparation.built_model_path),
            trained_model_path=Path(training.trained_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_patience= params.PATIENCE,
            params_factor= params.FACTOR,
            params_min_lr= params.MIN_LR,
            params_verbose= params.VERPOSE,
        )

        return training_config

In [6]:
class TrainModel:
    def __init__(self, config: TrainingConfig):
        # Initialize the training configuration
        self.config = config

    def data_generator(self):
        """
        Load and preprocess the training and validation data.

        Returns:
            train_data, val_data: The training and validation data generators.
        """
        # Implement data loading and preprocessing:
        train_datagen = ImageDataGenerator(
            rescale=1./255,               # Rescale pixel values to [0, 1]
            rotation_range=10,            # Rotate images randomly by up to 10 degrees
            width_shift_range=0.1,        # Shift images horizontally by up to 10% of the width
            zoom_range=0.1,               # Zoom in or out by up to 10%
            horizontal_flip=True,         # Flip images horizontally (left to right)
            )
        
        self.train_generator = train_datagen.flow_from_directory(
            f'{self.config.training_data}/train',
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='sparse',
            shuffle=True,
            )

        validation_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=10,
            horizontal_flip=True)
            
        self.validation_generator = validation_datagen.flow_from_directory(
            f'{self.config.training_data}/val',
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='sparse',
            shuffle=False,
            )
    
    
    def calculate_class_weights(self):
        """
        Calculate class weights to handle class imbalance.

        Args:
            train_generator: Training data generator.

        Returns:
            dict: Class weights.
        """
        y_train = self.train_generator.classes
        class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
        self.class_weights = dict(enumerate(class_weights))


    def get_built_model(self):
        # Load the built model from the specified path
        self.model = tf.keras.models.load_model(
            self.config.built_model_path
        )

    def train_model(self):
        """
        Train the model using the loaded data and specified parameters.
        Save the trained model to the specified path.
        """
        # Define the learning rate annealer
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            patience=self.config.params_patience,
            verbose=self.config.params_verbose,
            factor=self.config.params_factor,
            min_lr=self.config.params_min_lr
        )

        # Calculate steps per epoch and validation steps
        steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        validation_steps = self.validation_generator.samples // self.validation_generator.batch_size

        # Train the model
        history = self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.validation_generator,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,
            callbacks=[reduce_lr],
            class_weight=self.class_weights
        )

        # Save the trained model to the specified path
        self.save_model(path=self.config.trained_model_path, model=self.model)


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        # Save the model to the specified path
        model.save(path)

In [7]:
try:
    # Initialize the configuration manager to load config and params files
    config = ConfigurationManager()
    
    # Get the training configuration from the configuration manager
    training_config = config.get_training_config()
    
    # Initialize the Training class with the training configuration
    training = TrainModel(config=training_config)
    
    # Load the built model
    training.get_built_model()
    
    # Prepare the training and validation data generators
    training.data_generator()

    training.calculate_class_weights()
    
    # Train the model
    training.train_model()

except Exception as e:
    # Raise any exceptions that occur during the process
    raise e

[2024-08-03 23:41:14,110: INFO: common: YAML file: configYaml\config.yaml loaded successfully]
[2024-08-03 23:41:14,121: INFO: common: YAML file: configYaml\params.yaml loaded successfully]
[2024-08-03 23:41:14,123: INFO: common: Created directory at: artifacts]
[2024-08-03 23:41:14,126: INFO: common: Created directory at: artifacts\training]
Found 3067 images belonging to 4 classes.
Found 437 images belonging to 4 classes.
Epoch 1/60
81/95 [========================>.....] - ETA: 2:26 - loss: 1.8941 - accuracy: 0.2180